<a href="https://colab.research.google.com/github/DMVP2/MathThesis/blob/main/Implementaci%C3%B3n_AES_David_Mauricio_Valoyes_Porras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AES**

### **Importación de módulos y librerías**

In [56]:
import numpy as np

### **Funciones generales**

In [57]:
# Producto escalar entre vectores con coeficientes en el campo GF(2)

def producto_punto_GF2(a, b):
    return sum(a[i] * b[i] for i in range(len(a))) % 2

# Operación XOR bit a bit entre vectores con coeficientes en el campo GF(2)

def xor(a, b):
    return [a[i] ^ b[i] for i in range(len(a))]

# Función para convertir un número a su representación binaria

def convertir_binario(n, bits = 4):
    return [int(b) for b in format(n, f'0{bits}b')]

### **S-Box reducida**

La S-Box es una tabla de sustitución no lineal utilizada en AES, que reemplaza $n$ bits de entrada por $n$ bits de salida. Específicamente AES opera con S-Box de $8$ bits de entrada y $8$ bits de salida. Para el ejemplo del proyecto se construyó la siguiente S-Box reducida que toma $4$ bits de entrada y retorna $4$ bits en su salida:


| X | R Binaria | S(X) | R Binaria |
|:-:|:---------:|:----:|:---------:|
| 0 |    0000   |   E  |    1110   |
| 1 |    0001   |   4  |    0100   |
| 2 |    0010   |   D  |    1101   |
| 3 |    0011   |   1  |    0001   |
| 4 |    0100   |   2  |    0010   |
| 5 |    0101   |   F  |    1111   |
| 6 |    0110   |   B  |    1011   |
| 7 |    0111   |   8  |    1000   |
| 8 |    1000   |   3  |    0011   |
| 9 |    1001   |   A  |    1010   |
| A |    1010   |   6  |    0110   |
| B |    1011   |   C  |    1100   |
| C |    1100   |   5  |    0101   |
| D |    1101   |   9  |    1001   |
| E |    1110   |   0  |    0000   |
| F |    1111   |   7  |    0111   |

In [58]:
# Para la construcción de la S-Box se utilizaron diccionarios nativos de Python en su estructura "llave: valor"

# Diccionario que representa la S-Box de forma hexadecimal donde la llave es la entrada y el valor es la salida

s_box_reducida = {
    0x0: 0xE, 0x1: 0x4, 0x2: 0xD, 0x3: 0x1,
    0x4: 0x2, 0x5: 0xF, 0x6: 0xB, 0x7: 0x8,
    0x8: 0x3, 0x9: 0xA, 0xA: 0x6, 0xB: 0xC,
    0xC: 0x5, 0xD: 0x9, 0xE: 0x0, 0xF: 0x7
}

# Diccionario que repreenta la S-Box en forma binaria donde la llave es la entrada y el valor es la salida

s_box_binario = {x: convertir_binario(s_box_reducida[x]) for x in s_box_reducida}

print("\nS-Box reducida (S-Box de 4 bits)")
print("\n")
print("Entrada (hex) | Salida (hex) | Entrada (bin)   | Salida (bin)")
print("-" * 65)

# Impresión de la S-Box en forma hexadecimal y binaria
''' Para imprimir la tabla se recorre el diccionario de la S-Box en bluce (desde el 0 hasta el 15) tomando la llave o el
valor según corresponda y luego formatea la respectiva posición del diccionario correspondiente (s_box_reducida o s_box_binario).
En el caso de la columnas en hexadecimal tambien formatea a mayúscula'''

for x in s_box_reducida:
    print(f"       {x:1X}      |       {s_box_reducida[x]:1X}      |   {convertir_binario(x)}  | {s_box_binario[x]}")


S-Box reducida (S-Box de 4 bits)


Entrada (hex) | Salida (hex) | Entrada (bin)   | Salida (bin)
-----------------------------------------------------------------
       0      |       E      |   [0, 0, 0, 0]  | [1, 1, 1, 0]
       1      |       4      |   [0, 0, 0, 1]  | [0, 1, 0, 0]
       2      |       D      |   [0, 0, 1, 0]  | [1, 1, 0, 1]
       3      |       1      |   [0, 0, 1, 1]  | [0, 0, 0, 1]
       4      |       2      |   [0, 1, 0, 0]  | [0, 0, 1, 0]
       5      |       F      |   [0, 1, 0, 1]  | [1, 1, 1, 1]
       6      |       B      |   [0, 1, 1, 0]  | [1, 0, 1, 1]
       7      |       8      |   [0, 1, 1, 1]  | [1, 0, 0, 0]
       8      |       3      |   [1, 0, 0, 0]  | [0, 0, 1, 1]
       9      |       A      |   [1, 0, 0, 1]  | [1, 0, 1, 0]
       A      |       6      |   [1, 0, 1, 0]  | [0, 1, 1, 0]
       B      |       C      |   [1, 0, 1, 1]  | [1, 1, 0, 0]
       C      |       5      |   [1, 1, 0, 0]  | [0, 1, 0, 1]
       D      |       9      |

### **Tabla de aproximaciones lineales (LAT)**

La tabla LAT consiste en una tabla que compendia el resultado de evaluar cuantas veces se cumplen todas las posibles relaciones lineales (menos 8), es decir la cantidad de veces que todas las posibles ecuaciones de la forma $\sum_{k \, \in \, S_{1}} \, x_{k} \oplus \sum_{l \, \in \, S_{2}} \, y_{l} = 0$ dados $n$ bits de entrada y $n$ bits de salida se cumplen (menos 8).

El número de bits de entrada define la cantidad de posibles máscaras de entrada y el número de bits de salida define la cantidad de posibles máscaras de salida. En general se tienen $2^{n}$ máscaras siendo $n$ la cantidad de bits de entrada o de salida. Luego el tamaño de la tabla va a ser igual a la cantidad de máscaras de entrada multiplicada por la cantidad de máscaras de salida. En general el tamaño de la tabla será $2^{2n}$.

In [59]:
# La LAT muestra qué tan bien se aproxima una relación lineal entre bits de entrada y bits de salida

def construir_tabla_LAT():
    LAT = np.zeros((16, 16), dtype=int) # La tabla las es de 16 x 16 puesto que hay 4 bits de entrada y 4 bits de salida
    for alpha in range(16): # Todas las máscaras de entrada (4 bits)
        for beta in range(16): # Todas las máscaras de salida (4 bits)
            count = 0
            for x in range(16): # Probar todas las entradas posibles
                x_bits = convertir_binario(x)
                y_bits = s_box_binario[x]
                if producto_punto_GF2(convertir_binario(alpha), x_bits) == producto_punto_GF2(convertir_binario(beta), y_bits): # Verificar si se cumple α·x = β·y
                  count += 1
            LAT[alpha][beta] = count - 8  # Valor centrado
    return LAT

LAT = construir_tabla_LAT()

print("\nTabla de aproximaciones lineales (LAT)")
print("\n")
print("La posición x[i][j] corresponde al número de veces que una relación lineal se cumple menos 8")
print("\n")
print("\n   |  0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F")
print("-" * 55)

# Impresión en bucle de las filas de la tabla LAT

for alpha in range(16):
    print(f" {alpha:1X} | " + " ".join(f"{LAT[alpha][beta]:2}" for beta in range(16))) # El join permite realizar un bucle interno que imprime las columnas de cada fila

# Ejemplo: Sesgo para α = A (1010) y β = 9 (1001)

alpha_ejemplo = 0xA
beta_ejemplo = 0x9
sesgo = (LAT[alpha_ejemplo][beta_ejemplo]) / 16
probabilidad = (1 - sesgo) * 100

print("\nSegún la relación lineal con el sesgo mas alto se define α = i, β = j")

print(f"\nEjemplo: Para α = {alpha_ejemplo:X}, β = {beta_ejemplo:X}:")
print(f"- N° de cumplimientos: {LAT[alpha_ejemplo][beta_ejemplo] + 8}/16")
print(f"- Probabilidad de que la relación lineal se cumpla: {probabilidad}%")
print(f"- Sesgo: {sesgo:.3f}") # Se imprime el sesgo con tres decimales


Tabla de aproximaciones lineales (LAT)


La posición x[i][j] corresponde al número de veces que una relación lineal se cumple menos 8



   |  0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F
-------------------------------------------------------
 0 |  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1 |  0  0 -2 -2  0  0 -2  6  2  2  0  0  2  2  0  0
 2 |  0  0 -2 -2  0  0 -2 -2  0  0  2  2  0  0 -6  2
 3 |  0  0  0  0  0  0  0  0  2 -6 -2 -2  2  2 -2 -2
 4 |  0  2  0 -2 -2 -4 -2  0  0 -2  0  2  2 -4  2  0
 5 |  0 -2 -2  0 -2  0  4  2 -2  0 -4  2  0 -2 -2  0
 6 |  0  2 -2  4  2  0  0  2  0 -2  2  4 -2  0  0 -2
 7 |  0 -2  0  2  2 -4  2  0 -2  0  2  0  4  2  0  2
 8 |  0  0  0  0  0  0  0  0 -2  2  2 -2  2 -2 -2 -6
 9 |  0  0 -2 -2  0  0 -2 -2 -4  0 -2  2  0  4  2 -2
 A |  0  4 -2  2 -4  0  2 -2  2  2  0  0  2  2  0  0
 B |  0  4  0 -4  4  0  4  0  0  0  0  0  0  0  0  0
 C |  0 -2  4 -2 -2  0  2  0  2  0  2  4  0  2  0 -2
 D |  0  2  2  0 -2  4  0  2 -4 -2  2  0  2  0  0  2
 E |  0  2 

### **Generación de pares $(P,C)$**

Sea $K$ una clave de cifrado de $n$ bits, y sea $P$ un mensaje a cifrar, en el proceso de cifrado de AES primero se opera mediante XOR el bloque de datos con la clave de cifrado. Dado que se está trabajando con una sola ronda de encriptación el bloque de datos consiste en el mensaje en texto plano.

Así entonces, dado el conjunto de ejemplo de mensajes en texto plano $P = \{0000, 0001, 0010, 0011, 0100\}$, cada elemento del conjunto se opera con la clave de cifrado $K$ mediante XOR y luego ese resultado es enviado a su imagen según lo establecido por la S-Box.

In [60]:
# Se simula un cifrado AES de una sola ronda: C = S(P ⊕ K)

def cifrar(P, K_binario):
    P_xor_K = xor(P, K_binario) # Se opera mediante XOR el bloque de datos con la clave de cifrado
    P_xor_K_decimal = int(''.join(map(str, P_xor_K)), 2) # Convierte cada resultado de binario a decimal (el resultado será un número entre 0 y 15)
    C_decimal = s_box_reducida[P_xor_K_decimal] # Utiliza la representación decimal del binario para buscar la imagen correspondiente por medio de la S-Box (busca la posición correspondiente de 0 a 15)
    return convertir_binario(C_decimal)

# Clave de cifrado (K = 1011)

K_binario = [1, 0, 1, 1]
K_hex = int(''.join(map(str, K_binario)), 2)

# Conjunto de texto plano de ejemplo

P_muestra = [
    [0, 0, 0, 0],  # 0x0
    [0, 0, 0, 1],  # 0x1
    [0, 0, 1, 0],  # 0x2
    [0, 0, 1, 1],  # 0x3
    [0, 1, 0, 0],  # 0x4
]

# Generación de pares (P, C)

pares_P_C = [] # Lista de pares (P, C)
for P in P_muestra:
    C = cifrar(P, K_binario) # Se utiliza la función de cifrado para generar los pares (P, C)
    pares_P_C.append((P, C)) # Anexa cada par (P, C) a la lista

# Mostrar los pares generados
print("\nPares (P, C) GENERADOS")
print("\n")
print("      P      |    P ⊕ K    | C = S(P ⊕ K)")
print("-" * 45)

# Impresión del resultado C = S(P ⊕ K) de la ronda de cifrado mostrando los dos pasos P ⊕ K y C = S(P ⊕ K)

for P, C in pares_P_C:
    P_xor_K = xor(P, K_binario)
    print(f"{P} | {P_xor_K} | {C}")


Pares (P, C) GENERADOS


      P      |    P ⊕ K    | C = S(P ⊕ K)
---------------------------------------------
[0, 0, 0, 0] | [1, 0, 1, 1] | [1, 1, 0, 0]
[0, 0, 0, 1] | [1, 0, 1, 0] | [0, 1, 1, 0]
[0, 0, 1, 0] | [1, 0, 0, 1] | [1, 0, 1, 0]
[0, 0, 1, 1] | [1, 0, 0, 0] | [0, 0, 1, 1]
[0, 1, 0, 0] | [1, 1, 1, 1] | [0, 1, 1, 1]


### **Cálculo de la relación lineal $\alpha \cdot P \oplus \beta \cdot C = \alpha \cdot K$**

Se aplica la relación lineal $\alpha \, \cdot P \oplus \beta \, \cdot C = \alpha \, \cdot K$ con $\alpha = A = 1010$ y $\beta = 9 = 1001$ gracias a que corresponde a una de las relaciones lineales de la tabla LAT con mayor probabilidad. Es decir que el valor de $\alpha \, \cdot K$ se determinará mediante el resultado de calcular $\alpha \, \cdot P \oplus \beta \, \cdot C$.

In [61]:
# Se utiliza α = A (1010) y β = 9 (1001)

alpha = [1, 0, 1, 0]  # 0xA (1010)
beta = [1, 0, 0, 1]    # 0x9 (1001)

# Se calcula α·P ⊕ β·C para cada par (P, C)

resultados_alpha_P_beta_C = [] # Lista de resultados de calcular α·P ⊕ β·C para cada par (P, C)
for P, C in pares_P_C:
    term = producto_punto_GF2(alpha, P) ^ producto_punto_GF2(beta, C) # Cálculo de α·P ⊕ β·C para cada par (P, C)
    resultados_alpha_P_beta_C.append(term)

print("\nResultados de la relación lineal α·P ⊕ β·C")
print("\n")
print(f"Con α = {alpha_ejemplo:X}, β = {beta_ejemplo:X}:")
print("\n")
print("Para cada par (P, C), se calcula α·P ⊕ β·C:")
print("\n")

# Impresión de cada resultado

for i, (P, C) in enumerate(pares_P_C):
    print(f"P = {P}, C = {C} → α·P ⊕ β·C = {resultados_alpha_P_beta_C[i]}")


Resultados de la relación lineal α·P ⊕ β·C


Con α = A, β = 9:


Para cada par (P, C), se calcula α·P ⊕ β·C:


P = [0, 0, 0, 0], C = [1, 1, 0, 0] → α·P ⊕ β·C = 1
P = [0, 0, 0, 1], C = [0, 1, 1, 0] → α·P ⊕ β·C = 0
P = [0, 0, 1, 0], C = [1, 0, 1, 0] → α·P ⊕ β·C = 0
P = [0, 0, 1, 1], C = [0, 0, 1, 1] → α·P ⊕ β·C = 0
P = [0, 1, 0, 0], C = [0, 1, 1, 1] → α·P ⊕ β·C = 1


### **Evaluación de las claves candidatas**

Dado que la clave es de $4$ bits, existen $16$ claves $(2^{4})$ candidatas que corresponden a todas las posibles combinaciones de bits en $0$ y bits en $1$ en un arreglo de $4$ bits, por lo cual todas las posibles claves candidatas corresponden al conjunto: $\{0000, 0001, 0010, 0100, 1000, 1001, 1010, 1100, 0101, 0110, 0011, 1011, 1110, 0111, 1101, 1111\}$

Para cada clave candidata se calcula $\alpha \, \cdot K$, y luego se compara con el resultado de $\alpha \cdot P \oplus \beta \cdot C$, es decir se verifica cuantas veces se cumple que $\alpha \cdot P \oplus \beta \cdot C = \alpha \, \cdot K$

In [62]:
# Se evaluan todas las claves candidatas (16 posibles)

claves_candidatas = [convertir_binario(k) for k in range(16)] # Se convierten a binario todas las posibles claves candidatas
claves_candidatas_coincidencias = {} # Se define una lista que almacenará la cantidad de veces que se cumple que α·P ⊕ β·C = a·K

for candidata in claves_candidatas:
    alpha_K = producto_punto_GF2(alpha, candidata) # Cálculo de α·K
    coincidencias = sum(1 for resultado in resultados_alpha_P_beta_C if resultado == alpha_K) # Se suma 1 al resultado cada vez que se cumple que α·P ⊕ β·C = a·K
    claves_candidatas_coincidencias[int(''.join(map(str, candidata)), 2)] = coincidencias

print("\nCoincidencia del cálculo de α·K de las claves candidatas frente al cálculo α·K de la relación lineal α·P ⊕ β·C = α·K")
print("\n")
print("Clave (bin)  | Coincidencias/5")
print("-" * 30)

# Impresión de la cantidad de veces que u

for key, coincidencias in sorted(claves_candidatas_coincidencias.items(), key=lambda x: -x[1]):
    print(f"{convertir_binario(key)} |      {coincidencias}/5")


Coincidencia del cálculo de α·K de las claves candidatas frente al cálculo α·K de la relación lineal α·P ⊕ β·C = α·K


Clave (bin)  | Coincidencias/5
------------------------------
[0, 0, 0, 0] |      3/5
[0, 0, 0, 1] |      3/5
[0, 1, 0, 0] |      3/5
[0, 1, 0, 1] |      3/5
[1, 0, 1, 0] |      3/5
[1, 0, 1, 1] |      3/5
[1, 1, 1, 0] |      3/5
[1, 1, 1, 1] |      3/5
[0, 0, 1, 0] |      2/5
[0, 0, 1, 1] |      2/5
[0, 1, 1, 0] |      2/5
[0, 1, 1, 1] |      2/5
[1, 0, 0, 0] |      2/5
[1, 0, 0, 1] |      2/5
[1, 1, 0, 0] |      2/5
[1, 1, 0, 1] |      2/5


### **Filtrado de las claves mas propables basados en el sesgo obtenido**

Se seleccionan aquellas claves que tengan una mayor cantidad de coincidencias, lo que es equivalente a decir que tienen el sesgo mas alto puesto que su resultado luego de restar $1/2$ será mas alto

In [63]:
# Filtrar claves más probables (mayor cantidad de coincidencias)

claves_probables = [k for k, score in claves_candidatas_coincidencias.items() if score == max(claves_candidatas_coincidencias.values())] # Se seleccionan aquellas claves candidatas cuya cantidad de coincidencias sea igual al máximo de coincidencias

print("\nFiltrado de las claves mas probables")
print("\n")
for key in claves_probables:
    print(f"- {convertir_binario(key)} (hex {key:X})")

# Verificar si la clave verdadera está entre las candidatas

print(f"Clave verdadera: {K_binario} (hex {K_hex:X})")
print("\n")
print("¿Está entre las claves probables?", "Si" if K_hex in claves_probables else "No") # Se utiliza el operador ternario para verificar si la clave real está entre las claves candidatas mas probables


Filtrado de las claves mas probables


- [0, 0, 0, 0] (hex 0)
- [0, 0, 0, 1] (hex 1)
- [0, 1, 0, 0] (hex 4)
- [0, 1, 0, 1] (hex 5)
- [1, 0, 1, 0] (hex A)
- [1, 0, 1, 1] (hex B)
- [1, 1, 1, 0] (hex E)
- [1, 1, 1, 1] (hex F)
Clave verdadera: [1, 0, 1, 1] (hex B)


¿Está entre las claves probables? Si
